# Hello, TensorFlow!

* Names and execution in Python and TensorFlow
* The simplest TensorFlow graph
* The simplest TensorFlow neuron
* See your graph in TensorBoard
* Making the neuron learn
* Training diagnostics in TensorBoard
* Flowing onward

*See the [O'Reilly post](https://www.oreilly.com/learning/hello-tensorflow) for more explanatory text.*

---

### Names and execution in Python and TensorFlow

![An object has no name.](img/an_object_has_no_name.jpg)

*Image courtesy of [Hadley Wickham](https://twitter.com/hadleywickham/status/732288980549390336).*

In [1]:
foo = []

In [2]:
bar = foo

In [3]:
foo == bar

True

In [4]:
foo is bar

True

In [5]:
id(foo)

4359366272

In [6]:
id(bar)

4359366272

In [7]:
foo.append(bar)

In [8]:
foo

[[...]]

![foo points to itself](img/foo_loop.png)

*Image made with [draw.io](https://draw.io/).*

---

### The simplest TensorFlow graph

In [9]:
import tensorflow as tf

In [10]:
graph = tf.get_default_graph()

In [11]:
graph.get_operations()

[]

In [12]:
input_value = tf.constant(1.0)

In [13]:
operations = graph.get_operations()

In [14]:
operations

In [15]:
operations[0].node_def

name: "Const"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_FLOAT
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_FLOAT
      tensor_shape {
      }
      float_val: 1.0
    }
  }
}

In [16]:
input_value

<tf.Tensor 'Const:0' shape=() dtype=float32>

In [17]:
sess = tf.Session()

In [18]:
sess.run(input_value)

1.0

### The simplest TensorFlow neuron

In [19]:
weight = tf.Variable(0.8)

In [20]:
for op in graph.get_operations(): print(op.name)

Const
Variable/initial_value
Variable
Variable/Assign
Variable/read


(The parentheses for `print` are necessary for Python 3 but optional for Python 2.)

In [21]:
output_value = weight * input_value

In [22]:
op = graph.get_operations()[-1]

In [23]:
op.name

u'mul'

In [24]:
for op_input in op.inputs: print(op_input)

Tensor("Variable/read:0", shape=(), dtype=float32)
Tensor("Const:0", shape=(), dtype=float32)


In [25]:
init = tf.initialize_all_variables()

In [26]:
sess.run(init)

In [27]:
sess.run(output_value)

0.80000001

This is the neuron's "inference" or "forward pass".

---

### See your graph in TensorBoard

In [28]:
tf.reset_default_graph()
sess = tf.Session()
!rm -rf log_simple_graph
!rm -rf log_simple_stat

In [29]:
x = tf.constant(1.0, name='input')
w = tf.Variable(0.8, name='weight')
y = tf.mul(w, x, name='output')

In [30]:
summary_writer = tf.train.SummaryWriter('log_simple_graph', sess.graph)

In [31]:
!tensorboard --logdir=log_simple_graph

Starting TensorBoard 16 on port 6006
(You can navigate to http://0.0.0.0:6006)
127.0.0.1 - - [29/Jun/2016 16:35:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:35:05] "GET /lib/css/global.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:35:05] "GET /external/lodash/lodash.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:35:05] "GET /external/d3/d3.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:35:05] "GET /external/plottable/plottable.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:35:05] "GET /external/plottable/plottable.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:35:05] "GET /external/graphlib/dist/graphlib.core.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:35:05] "GET /external/dagre/dist/dagre.core.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:35:05] "GET /external/webcomponentsjs/webcomponents-lite.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:35:05] "GET /external/polymer/polymer.html HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 

After startup, go to http://localhost:6006/#graphs to see the interface.

To continue with the notebook, interrupt the kernel by using the square "stop" button or by typing `esc`, `i`, `i`.

### Making the neuron learn

In [32]:
y_ = tf.constant(0.0)

In [33]:
loss = (y - y_)**2

In [34]:
optim = tf.train.GradientDescentOptimizer(learning_rate=0.025)

In [35]:
grads_and_vars = optim.compute_gradients(loss)

![plot of loss and derivative](img/plot.png)

In [36]:
sess.run(tf.initialize_all_variables())

In [37]:
sess.run(grads_and_vars[0][0])

1.6

We have `y = 0.8` and `y_ = 0`.

So the loss is `(0.8 - 0)^2 = 0.64`.

And the derivative of the loss is `2*(0.8 - 0)`.

In [38]:
sess.run(optim.apply_gradients(grads_and_vars))

We have now completed a forward and backward pass.

In [39]:
sess.run(w)

0.75999999

This comes from subtracting the learning rate times the derivative of the loss:

`0.8 - 0.025*1.6 = 0.8 - 0.04 = 0.76`

In [40]:
train_step = tf.train.GradientDescentOptimizer(0.025).minimize(loss)

In [41]:
for i in range(100):
    sess.run(train_step)

In [42]:
sess.run(y)

0.0044996012

### Training diagnostics in TensorBoard

In [43]:
sess.run(tf.initialize_all_variables())

In [44]:
for i in range(100):
    print('before step {}, y is {}'.format(i, sess.run(y)))
    sess.run(train_step)

before step 0, y is 0.800000011921
before step 1, y is 0.759999990463
before step 2, y is 0.722000002861
before step 3, y is 0.685899972916
before step 4, y is 0.651604950428
before step 5, y is 0.619024693966
before step 6, y is 0.588073432446
before step 7, y is 0.558669745922
before step 8, y is 0.530736267567
before step 9, y is 0.504199445248
before step 10, y is 0.478989481926
before step 11, y is 0.45504000783
before step 12, y is 0.432288020849
before step 13, y is 0.410673618317
before step 14, y is 0.390139937401
before step 15, y is 0.370632946491
before step 16, y is 0.352101296186
before step 17, y is 0.334496229887
before step 18, y is 0.317771404982
before step 19, y is 0.301882833242
before step 20, y is 0.286788702011
before step 21, y is 0.27244925499
before step 22, y is 0.25882679224
before step 23, y is 0.245885446668
before step 24, y is 0.233591169119
before step 25, y is 0.221911609173
before step 26, y is 0.210816025734
before step 27, y is 0.200275227427
befor

In [45]:
summary_y = tf.scalar_summary('output', y)

In [46]:
summary_writer = tf.train.SummaryWriter('log_simple_stat')

In [47]:
sess.run(tf.initialize_all_variables())
for i in range(100):
    summary_str = sess.run(summary_y)
    summary_writer.add_summary(summary_str, i)
    sess.run(train_step)

In [48]:
!tensorboard --logdir=log_simple_stat

Starting TensorBoard 16 on port 6006
(You can navigate to http://0.0.0.0:6006)
127.0.0.1 - - [29/Jun/2016 16:36:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:08] "GET /external/lodash/lodash.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:08] "GET /lib/css/global.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:08] "GET /external/d3/d3.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:08] "GET /external/plottable/plottable.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:08] "GET /external/plottable/plottable.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:08] "GET /external/graphlib/dist/graphlib.core.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:08] "GET /external/dagre/dist/dagre.core.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:08] "GET /external/webcomponentsjs/webcomponents-lite.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:08] "GET /external/polymer/polymer.html HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 

After startup, go to http://localhost:6006/#events to see the interface.

To continue with the notebook, interrupt the kernel by using the square "stop" button or by typing `esc`, `i`, `i`.

---

### Flowing onward

In [49]:
tf.reset_default_graph()
!rm -rf log_simple_stats

The script below is designed to stand alone, and will also work here with the default graph reset.

In [50]:
import tensorflow as tf

x = tf.constant(1.0, name='input')
w = tf.Variable(0.8, name='weight')
y = tf.mul(w, x, name='output')
y_ = tf.constant(0.0, name='correct_value')
loss = tf.pow(y - y_, 2, name='loss')
train_step = tf.train.GradientDescentOptimizer(0.025).minimize(loss)

for value in [x, w, y, y_, loss]:
    tf.scalar_summary(value.op.name, value)

summaries = tf.merge_all_summaries()

sess = tf.Session()
summary_writer = tf.train.SummaryWriter('log_simple_stats', sess.graph)

sess.run(tf.initialize_all_variables())
for i in range(100):
    summary_writer.add_summary(sess.run(summaries), i)
    sess.run(train_step)

In [51]:
!tensorboard --logdir=log_simple_stats

Starting TensorBoard 16 on port 6006
(You can navigate to http://0.0.0.0:6006)
127.0.0.1 - - [29/Jun/2016 16:36:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:31] "GET /lib/css/global.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:31] "GET /external/lodash/lodash.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:31] "GET /external/d3/d3.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:31] "GET /external/plottable/plottable.css HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:31] "GET /external/plottable/plottable.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:31] "GET /external/graphlib/dist/graphlib.core.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:31] "GET /external/dagre/dist/dagre.core.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:31] "GET /external/webcomponentsjs/webcomponents-lite.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 16:36:31] "GET /external/polymer/polymer.html HTTP/1.1" 200 -
127.0.0.1 - - [29/Jun/2016 

After startup, go to http://localhost:6006/#events to see the interface.

To continue with the notebook, interrupt the kernel by using the square "stop" button or by typing `esc`, `i`, `i`.